# Multiforest optimization notebook

Above the code cells, there will be instructions how the users should modify the codes in the cells. If there are no instructions, then by default no changes should be needed for the cell.

## Read the data

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/py_class")

import multiFunctionalOptimizationSwe as MFO

In [2]:
from importlib import reload
reload(MFO)

<module 'multiFunctionalOptimizationSwe' from '/home/ubuntu/workspace/mf_optimization_demo/py_class/multiFunctionalOptimizationSwe.py'>

You can choose solver by defining "solver=XXX" in the argument. Possible options are "CPLEX", "CLP" and "GLOP"

In [3]:
mfo = MFO.MultiFunctionalOptimization(solver = "CLP")

'Using CLP'

Define CC scenario data

In [4]:
RCP = "RCP0"
objectives_globiom = 'globiom_RCP0_V1'

In [5]:
#RCP = "RCP45"
#objectives_globiom = 'globiom_RCP45_V1'

1.5 degrees = RCP0 (no CC), NDC = RCP 4.5

In [6]:
scenario = "MF"

In [7]:
import wget
import os
import numpy as np
import pandas as pd

NoCC:

In [8]:
%%time
mfo.readData("Sweden_5%_RCP0.zip",
             standsEnu = "Description", regimesEnu = ["ControlCategoryName", "AlternativeNo"], timeEnu = "period",
             sampleRatio = 1, #If no sample ratio given, the ratio is assumed to be 1
             areaCol = "RepresentedArea"
            )

CPU times: user 2.75 s, sys: 740 ms, total: 3.49 s
Wall time: 3.48 s


RCP45:

In [9]:
#%%time
#mfo.readData("Sweden_5%_RCP45.csv",
#             standsEnu = "Description", regimesEnu = ["ControlCategoryName", "AlternativeNo"], timeEnu = "period",
#             sampleRatio = 1,
#             areaCol = "RepresentedArea"
#            )

Remove forestry regimes connected to intensification:

In [10]:
indexNames = mfo.data[(mfo.data['ControlCategoryName'] == 'Int_Prod') | (mfo.data['ControlCategoryName'] == 'Int_HybridExotic') | (mfo.data['ControlCategoryName'] == 'Int_Contorta')].index
mfo.data.drop(indexNames, inplace = True)

Create a column for Pulpfuel i.e. pulp plus firewood

In [11]:
mfo.data['PulpFuel'] = mfo.data.SumPulpVolumeTotal.values + mfo.data.SumHarvestFuelwoodTotal.values

Create column for simulated harvest for globiom optimisation

In [12]:
mfo.data['SimulatedSAWlog'] = mfo.data.SumTimberVolumeTotal.values/5

In [13]:
mfo.data['SimulatedResidue'] = mfo.data.SumHarvestResiduesTotal.values/5

In [14]:
mfo.data['SimulatedPulPFuel'] = mfo.data.PulpFuel.values/5

Create a new column for old decidious forest: older than 80 years and more than 30% deciduous

In [15]:
mfo.data['DeciduousRatio'] = mfo.data.VolumeDecidous.values/mfo.data.StandingVolume.values

<ipython-input-15-9487c82f8522>:1: RuntimeWarning: invalid value encountered in true_divide
  mfo.data['DeciduousRatio'] = mfo.data.VolumeDecidous.values/mfo.data.StandingVolume.values


In [16]:
mfo.data["old_deciduous_rich_forest_area"] = (mfo.data['DeciduousRatio'].values>0.3)*(mfo.data["Age"].values>80)*mfo.data["RepresentedArea"].values

Create bolean indicator for set asides to calculate share of forest set aside from management,
for managed forest to calculate values only for managed forest, and for CCF to calculate share of CCF

In [17]:
mfo.data['SetAside'] = np.where(mfo.data.ControlCategoryName == 'SetAside (Unmanaged)', True, False)

In [18]:
mfo.data['managed'] = np.where(mfo.data.ControlCategoryName == 'SetAside (Unmanaged)', False, True)

In [19]:
mfo.data['CCF'] = np.where(mfo.data.ControlCategoryName == 'CCF', True, False)

Manual calculation of Total values:

In [20]:
mfo.data["Total_VolumeDeciduous"] = mfo.data["VolumeDecidous"] * mfo.data["RepresentedArea"]
mfo.data["Total_DeadWoodVolume"] = mfo.data["DeadWoodVolume"] * mfo.data["RepresentedArea"]
mfo.data["Total_RecreationIndex"] = mfo.data["RecreationIndex"] * mfo.data["RepresentedArea"]
mfo.data["Total_TotalCarbon"] = mfo.data["TotalCarbon"] * mfo.data["RepresentedArea"]

Set indicator values for BD indicators to 0 on set asides to only calculate with values on managed

In [21]:
mfo.data["Total_VolumeDeciduous_managed"] = mfo.data["Total_VolumeDeciduous"] * mfo.data["managed"]
mfo.data["Total_DeadWoodVolume_managed"] = mfo.data["Total_DeadWoodVolume"] * mfo.data["managed"]
mfo.data["old_deciduous_rich_forest_area_managed"] = mfo.data["old_deciduous_rich_forest_area"] * mfo.data["managed"]

## Finalise data:

In [22]:
mfo.finalizeData(initialTime=0, initialRegime="ControlCategoryNameInitial state_AlternativeNo1")

## GLOBIOM demands VERSION 1
E.g., used for "bottom up" approach, on top of national scenario optimizations, no assortment transfer possible.

In [23]:
# --------------
# 1.5 degree scenario; matches with RCP0 (no CC)
# --------------

if objectives_globiom == 'globiom_RCP0_V1':
    
    # read the csv with the globiom demands 
    # data was created in R (package "zoo")to extend the time series of globiom that they match with each simulation step
    demands1p5 = pd.read_csv("G1p5_5%.csv") 

    # extract individual columns and turn them into a list that goes into the objective problem
    sawlog1p5 = demands1p5["GSawlog_uB"]
    sawlog1p5 = sawlog1p5.to_list()
    
    pulpfuel1p5 = demands1p5["GPulpFuel_uB"]
    pulpfuel1p5 = pulpfuel1p5.to_list()

    residues1p5 = demands1p5["GResidues"]
    residues1p5 = residues1p5.to_list()
    
    GLOBdemand_RCP0 = {
        
    # Maximize Sawlog according to GLOBIOM demand at RCP1.5 scenario
    "GSawlog_RCP0":("Sawlog harvest levels according to GLOBIOM at RCP0",
                    "SimulatedSAWlog","max","periodicTargets","areaWeightedSum", sawlog1p5),
        
    # Maximize pulp wood according to GLOBIOM demand at RCP1.5 scenario
    "GPulpFuel_RCP0":("Pulp harvest levels according to GLOBIOM at RCP0",
                      "SimulatedPulPFuel","max","periodicTargets","areaWeightedSum", pulpfuel1p5),
        
    # Maximize residues according to GLOBIOM demand at RCP1.5 scenario
    "GResidues_RCP0":("Residues levels according to GLOBIOM at RCP0",
                       "SimulatedResidue","max","periodicTargets","areaWeightedSum", residues1p5)
        
    }
    
    print("used RCP0") 
    
elif objectives_globiom == 'globiom_RCP45_V1':
    
    demands4p5 = pd.read_csv("G4p5_5%.csv") 

    sawlog4p5 = demands4p5["GSawlog_uB"]
    sawlog4p5 = sawlog4p5.to_list()
    
    pulpfuel4p5 = demands4p5["GPulpFuel_uB"]
    pulpfuel4p5 = pulpfuel4p5.to_list()

    residues4p5 = demands4p5["GResidues"]
    residues4p5 = residues4p5.to_list()
    
    GLOBdemand_RCP45 = {
        
    # Maximize Sawlog according to GLOBIOM demand at RCP4.5 scenario
    "GSawlog_RCP45":("Sawlog harvest levels according to GLOBIOM at RCP4.5",
                     "SimulatedSAWlog","max","periodicTargets","areaWeightedSum", sawlog4p5),
        
    # Maximize pulp wood according to GLOBIOM demand at RCP4.5 scenario
    "GPulpFuel_RCP45":("PulpFuel harvest levels according to GLOBIOM at RCP4.5",
                       "SimulatedPulPFuel","max","periodicTargets","areaWeightedSum", pulpfuel4p5),
   
    # Maximize residues according to GLOBIOM demand at RCP4.5 scenario
    "GResidues_RCP45":("Residues levels according to GLOBIOM at RCP4.5",
                       "SimulatedResidue","max","periodicTargets","areaWeightedSum", residues4p5)
    
    }
    print("used RCP45")

else:
    globiom = {}
    print("no demands considered")

used RCP0


## Start defining the optimization problem

#### Define objectives

Objective format: Unique_key : (Long human readable name,column name in data, max/min objective, year wise aggregation, stand wise aggregation [, target year])

Options for "objective": "max"imise or "min"imise it
year wise aggregation: "min" (minimum value), "average", "firstYear", "sum", "targetYearWithSlope","targetYear"
stand wise aggregation: "sum", "areaWeightedAverage", "areaWeightedSum"
targe yeart: any year except the first one

Objective dictionary structure: "Unique short name":("Long human readable name","column name in the data")

In [24]:
Wood = {
"NetPresentValue":("Total sum net present value of cut forest","NPV","max","firstYear","areaWeightedSum"),
"TotalAnnIncrement":("Annual Increment (maximised min over all years)","AnnualIncrementNetTotal","max","min","areaWeightedAverage"),
"HarvestEvenFlow":("Average harvest","SumVolumeCutTotal","max","min","areaWeightedAverage")
}

In [25]:
Recreation = { 
"RecreationIndex":("No decrease in recreation index","Relative_Total_RecreationIndex","max","min","sum")
}

In [26]:
Climate = { 
"TotalCarbon":("No decrease carbon stocks","Relative_Total_TotalCarbon","max","min","sum")
}

In [27]:
Biodiversity = { 
"DeadWoodVolume":("60% incr in deadwood by 2050","Relative_Total_DeadWoodVolume_managed","max","targetYearWithSlope","sum",8),  
"OldDeciduous":("60% incr in old deciduous area by 2050","Relative_old_deciduous_rich_forest_area_managed","max","targetYearWithSlope","sum",8),
"SetAside":("Share of set aside forest","SetAside","max","firstYear","areaWeightedAverage")
}

In [28]:
Resilience = {
"DeciduousVolume":("60% incr in deciduous volume by 2050","Relative_Total_VolumeDeciduous_managed","max","targetYearWithSlope","sum",8)
}

In [29]:
Water = {
"CCF":("Share of CCF","CCF","max","firstYear","areaWeightedAverage")
}

In [30]:
# OBS! Switch to correct Globiom demand here!!!!!!!!!!!!!!!!!!!!!!!!!!!!
objectives = {
    **GLOBdemand_RCP0,
    #**GLOBdemand_RCP45,
    **Wood,
    **Recreation,
    **Climate,
    **Biodiversity,
    **Resilience,
    **Water
}

In [31]:
mfo.defineObjectives(objectives)

'Defining objectives'

'Aggregating stand wise'

100%|██████████| 13/13 [02:19<00:00, 10.70s/it]


'Aggregating year wise'

100%|██████████| 13/13 [00:00<00:00, 3570.09it/s]


'Objectives added'

## Calculate objective ranges

In [32]:
%%time
mfo.calculateObjectiveRanges()

'Calculating objective ranges'

  0%|          | 0/13 [00:00<?, ?it/s]

'Optimizing for Sawlog harvest levels according to GLOBIOM at RCP0'

'Found an optimal solution in 16 seconds'

'Objective values are:'

...

'60% incr in deciduous volume by 2050'

0.0

'Share of CCF'

0.3179936145437956

100%|██████████| 13/13 [01:10<00:00,  5.39s/it]
CPU times: user 1min 8s, sys: 2.26 s, total: 1min 10s
Wall time: 1min 10s


In [33]:
mfo.objectiveRanges

{'GSawlog_RCP0': (-1.0, -0.10866831012096011),
 'GPulpFuel_RCP0': (-1.0, 0.09989027454200498),
 'GResidues_RCP0': (-1.0, 0.8760084078485709),
 'NetPresentValue': (-23884342.86, 49463009181.49912),
 'TotalAnnIncrement': (0.0, 4.5654712241197934),
 'HarvestEvenFlow': (0.0, 24.506476009065032),
 'RecreationIndex': (0.0, 1.0350875091638725),
 'TotalCarbon': (0.0, 1.0342392601610315),
 'DeadWoodVolume': (-7.0, 0.18074308371731743),
 'OldDeciduous': (-7.0, 0.7202743451573959),
 'SetAside': (0.035147904804680205, 0.9999999999999979),
 'DeciduousVolume': (-7.0, 1.1549438102817142),
 'CCF': (0.0, 0.3179936145437956)}

## Export the objetive ranges

Can save re-calculation times if big data sets are optimised

In [34]:
import json
mfo.objectiveRanges

with open("objectiveRanges_V1_"+RCP+"_"+scenario+".json", "w") as json_file:
    json.dump(mfo.objectiveRanges, json_file)

Save the objectives ranges also as CSV

In [35]:
import pandas
df = pandas.read_json("objectiveRanges_V1_"+RCP+"_"+scenario+".json")
df.to_csv("objectiveRanges_V1_"+RCP+"_"+scenario+".csv")

## Show the GUI

In [36]:
mfo.showGUI(debug=True)

interactive(children=(FloatSlider(value=-1.0, description='Sawlog harvest levels according to GLOBIOM at RCP0'…

interactive(children=(FloatSlider(value=-0.5543341550604801, description='Sawlog harvest levels according to G…

interactive(children=(Button(description='Change constraints', style=ButtonStyle()), Output()), _dom_classes=(…

Button(description='Print solution', style=ButtonStyle())

## Export solution data as csv

In [37]:
import os

b = []
c = []
for key in mfo.regimesDecision.keys():
    if mfo.regimesDecision[key].solution_value() > 0:
        b = b+ [(key[0],x, key[1]) for x in range(0,21)]
        c = c+ [(key[0],key[1],mfo.regimesDecision[key].solution_value())]
data2b = mfo.data.iloc[mfo.data.index.isin(b)]
data2b.to_csv("solution_alldata_V1_"+RCP+"_"+scenario+"_data.csv")
c1 = pd.DataFrame(c)
c1.to_csv("solution_V1_"+RCP+"_"+scenario+"_solutions_test.csv")

## Export objective values

The optimal solution for each objective.

In [39]:
with open("objectiveValues_V1_"+scenario+'_'+RCP+".csv","w") as file: 
    delim = "" 
    for objName in mfo.objectiveTypes.keys(): 
        file.write(delim+objName) 
        delim = "," 
    file.write("\n") 
    delim = "" 
    for objName in mfo.objectiveTypes.keys(): 
        file.write(delim+str(mfo.objective[objName].solution_value())) 
        delim = "," 
    file.write("\n")